### 1 - Import test data

In [1]:
import yfinance as yf
import pandas as pd

dataF = yf.download("EURUSD=X", start="2024-12-01", end="2024-12-15", interval='15m')
dataF.iloc[:,:]
dataF.Open.iloc
dataF

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,EURUSD=X,EURUSD=X,EURUSD=X,EURUSD=X,EURUSD=X
Datetime,,,,,
2024-12-02 00:00:00,1.054296,1.054741,1.053630,1.054741,0
2024-12-02 00:15:00,1.054296,1.054407,1.054074,1.054407,0
2024-12-02 00:30:00,1.053630,1.054519,1.053519,1.054519,0
2024-12-02 00:45:00,1.053963,1.053963,1.053519,1.053519,0
2024-12-02 01:00:00,1.053630,1.054185,1.053519,1.054074,0
...,...,...,...,...,...
2024-12-13 21:15:00,1.050531,1.050531,1.049869,1.049869,0
2024-12-13 21:30:00,1.050420,1.050420,1.050089,1.050310,0


### 2 - Define your signal function

In [2]:
def Engulfing_pattern(df):
    # Ensure we are dealing with scalar values
    open_price = df.Open.iloc[-1]
    close_price = df.Close.iloc[-1]
    previous_open = df.Open.iloc[-2]
    previous_close = df.Close.iloc[-2]

    # Bearish Pattern
    if (open_price > close_price and 
        previous_open < previous_close and 
        close_price < previous_open and 
        open_price >= previous_close):
        return 1

    # Bullish Pattern
    elif (open_price < close_price and 
          previous_open > previous_close and 
          close_price > previous_open and 
          open_price <= previous_close):
        return 2

    # No clear pattern
    else:
        return 0

In [5]:
signal = []
signal.append(0)
for i in range(1,len(dataF)):
    df = dataF[i-1:i+1]
    signal.append(Engulfing_pattern(df))
Engulfing_pattern(data)
dataF["signal"] = signal

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
dataF.signal.value_counts()
#dataF.iloc[:, :]

0    3527
2     178
1     172
Name: signal, dtype: int64

### 3 - Connect to the market and execute trades

In [ ]:
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

In [ ]:
from config import access_token, accountID
def get_candles(n):
    #access_token='XXXXXXX'#you need token here generated from OANDA account
    client = CandleClient(access_token, real=False)
    collector = client.get_collector(Pair.EUR_USD, Gran.M15)
    candles = collector.grab(n)
    return candles

candles = get_candles(3)
for candle in candles:
    print(float(str(candle.bid.o))>1)


True
True
True


In [ ]:
def trading_job():
    candles = get_candles(3)
    dfstream = pd.DataFrame(columns=['Open','Close','High','Low'])

    i=0
    for candle in candles:
        dfstream.loc[i, ['Open']] = float(str(candle.bid.o))
        dfstream.loc[i, ['Close']] = float(str(candle.bid.c))
        dfstream.loc[i, ['High']] = float(str(candle.bid.h))
        dfstream.loc[i, ['Low']] = float(str(candle.bid.l))
        i=i+1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    signal = signal_generator(dfstream.iloc[:-1,:])#

    # EXECUTING ORDERS
    #accountID = "XXXXXXX" #your account ID here
    client = API(access_token)

    SLTPRatio = 2.
    previous_candleR = abs(dfstream['High'].iloc[-2]-dfstream['Low'].iloc[-2])

    SLBuy = float(str(candle.bid.o))-previous_candleR
    SLSell = float(str(candle.bid.o))+previous_candleR

    TPBuy = float(str(candle.bid.o))+previous_candleR*SLTPRatio
    TPSell = float(str(candle.bid.o))-previous_candleR*SLTPRatio

    print(dfstream.iloc[:-1,:])
    print(TPBuy, "  ", SLBuy, "  ", TPSell, "  ", SLSell)
    signal = 2
    #Sell
    if signal == 1:
        mo = MarketOrderRequest(instrument="EUR_USD", units=-1000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)
    #Buy
    elif signal == 2:
        mo = MarketOrderRequest(instrument="EUR_USD", units=1000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

### 4 - Executing orders automatically with a scheduler

In [ ]:
trading_job()

#scheduler = BlockingScheduler()
#scheduler.add_job(trading_job, 'cron', day_of_week='mon-fri', hour='00-23', minute='1,16,31,46', start_date='2022-01-12 12:00:00', timezone='America/Chicago')
#scheduler.start()